# Habituation to turbulent backgrounds as a function of model rates

For IBCM, measure the robustness of alignment to one odor using, for each neuron, the difference between its maximum and second largest (or minimum?) alignments, or some similar metric. Then average across neurons to measure the full network convergence. 

For BioPCA, measure convergence to eigenvalues ? Or alignment subspace error as usual? 

Ideally, we would also measure new odor recognition performance? This would require a new simulation script and massive simulations, one ensemble per choice of $\tau_\Theta, \mu$ for IBCM, and similarly for BioPCA... 

Things to check:
 - Convergence of IBCM as as function of $\mu$ (or equivalently the OSN input amplitude? Or some moment of the background? not sure) and $\tau_\Theta$. 
 - Convergence as a function of the number of odors, the strength of turbulence
 
## Planning ahead
Try with full-fledged turbulent backgrounds. Possibly need to run multiple background seeds, average convergence for each choice of rates or background size/turbulence strength. 
 
If it gets too murky, use a simpler background: the outcomes were very clear for a 2D toy model background, perhaps with 3+ odors but milder fluctuations it is also clearer. Easier to tune the variance of a O-U background than the turbulent process. Also, would allow us to use the Intrator default model version, rather than the Law and Cooper modification, which speeds up convergence in a hardly predictable way. 

## Imports

In [ ]:
import numpy as np
from scipy import sparse, special
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from time import perf_counter
import os, json
from os.path import join as pj
import sys
if ".." not in sys.path:
    sys.path.insert(1, "..")


from modelfcts.ibcm import (
    integrate_inhib_ibcm_network_options,
    ibcm_respond_new_odors,
    compute_mbars_hgammas_hbargammas,
    ibcm_respond_new_odors
)
from modelfcts.ibcm_analytics import (
    fixedpoint_thirdmoment_exact, 
    ibcm_fixedpoint_w_thirdmoment, 
    ibcm_all_largest_eigenvalues
)
from modelfcts.biopca import (
    integrate_inhib_biopca_network_skip,
    build_lambda_matrix,
    biopca_respond_new_odors
)
from modelfcts.average_sub import (
    integrate_inhib_average_sub_skip, 
    average_sub_respond_new_odors
)
from modelfcts.ideal import (
    find_projector, 
    find_parallel_component, 
    ideal_linear_inhibitor, 
    compute_ideal_factor
)
from modelfcts.checktools import (
    analyze_pca_learning, 
    check_conc_samples_powerlaw_exp1
)
from modelfcts.backgrounds import (
    update_powerlaw_times_concs, 
    logof10, 
    sample_ss_conc_powerlaw, 
    generate_odorant
)
from utils.statistics import seed_from_gen
from modelfcts.distribs import (
    truncexp1_average,
    powerlaw_cutoff_inverse_transform
)
from utils.smoothing_function import (
    moving_average, 
    moving_var
)
from simulfcts.plotting import (
    plot_hbars_gamma_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition, 
    plot_pca_results, 
    hist_outline
)
from simulfcts.analysis import compute_back_reduction_stats
from utils.metrics import jaccard, l2_norm

# Initialization

### Aesthetic parameters

In [ ]:
do_save_plots = False
do_save_outputs = False

root_dir = pj("..")
outputs_folder = pj(root_dir, "results", "for_plots", "convergence")
panels_folder = pj(root_dir, "figures", "convergence")
params_folder = pj(root_dir, "results", "common_params")

# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full24 = np.asarray(json.load(f))
# Here, 32 neurons, need to make a new palette with same parameters
neuron_colors_full = np.asarray(sns.husl_palette(n_colors=32, h=0.01, s=0.9, l=0.4, as_cmap=False))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

# Background generation and initialization functions

In [ ]:
def linear_combi(concs, backs):
    """ concs: shaped [..., n_odors]
        backs: 2D array, shaped [n_odors, n_osn]
    """
    return concs.dot(backs)

In [ ]:
# Global choice of background and odor mixing functions
update_fct = update_powerlaw_times_concs
combine_fct = linear_combi

In [ ]:
# We will later explore the effect of varying these parameters on the convergence, 
# but put the default ones in a function
def default_background_params(n_comp):
    """ Default time and concentration parameters for the turbulent process"""
    # Turbulent background parameters: same rates and constants for all odors
    back_pms_turbulent = [
        np.asarray([1.0] * n_comp),        # whiff_tmins
        np.asarray([500.] * n_comp),       # whiff_tmaxs
        np.asarray([1.0] * n_comp),        # blank_tmins
        np.asarray([800.0] * n_comp),      # blank_tmaxs
        np.asarray([0.6] * n_comp),        # c0s
        np.asarray([0.5] * n_comp),        # alphas
    ]
    return back_pms_turbulent

In [ ]:
# Background initialization, given parameters and a seeded random generator
def initialize_given_background(back_pms, rgen, n_comp, n_dim):
    # Initial values of background process variables (t, c for each variable)
    init_concs = sample_ss_conc_powerlaw(*back_pms[:-1], size=1, rgen=rgen)
    init_times = powerlaw_cutoff_inverse_transform(
                    rgen.random(size=n_comp), *back_pms[2:4])
    tc_init = np.stack([init_times, init_concs.squeeze()], axis=1)

    # Initial background vector: combine odors with the tc_init concentrations
    back_comps = back_pms[-1]
    init_bkvec = combine_fct(tc_init[:, 1], back_comps)
    # background random variables are first in the list of initial values
    init_back = [tc_init, init_bkvec]
    
    return init_back

# IBCM simulation functions

In [ ]:
# Analyze to establish convergence to specific fixed points. 
def analyze_ibcm_simulation(sim_results, ibcm_rates_loc, back_pms, 
                            skp_loc=20, dt=1.0, duration_loc=360000.0):
    """
    Args:
        sim_results = (tser_ibcm, nuser_ibcm, bkvecser_ibcm, mser_ibcm, 
            hbarser_ibcm, thetaser_ibcm, wser_ibcm, yser_ibcm)
        ibcm_rates_loc: learnrate_ibcm, tau_avg_ibcm, coupling_eta_ibcm, ...
            
    Returns:
        alignment_gaps, indexed [neuron]
        specif_gammas, indexed [neuron]
        gamma_vari, indexed [neuron, component]
    """
    coupling = ibcm_rates_loc[2]
    (tser_ibcm, nuser_ibcm, bkvecser_ibcm, mser_ibcm, 
        hbarser_ibcm, thetaser_ibcm, wser_ibcm, yser_ibcm) = sim_results
    # Calculate hgammas_bar and mbars
    transient = int(5/6*duration_loc / dt) // skp_loc
    basis = back_pms[-1]
    
    # Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
    mbarser, c_gammas, hbars_gamma = compute_mbars_hgammas_hbargammas(mser_ibcm, coupling, basis)
    hbars_gamma_mean = np.mean(hbars_gamma[transient:], axis=0)
    # Sorted odor indices, from min to max, of odor alignments for each neuron
    aligns_idx_sorted = np.argsort(hbars_gamma_mean, axis=1) 
    specif_gammas = np.argmax(hbars_gamma_mean, axis=1)
    assert np.all(specif_gammas == aligns_idx_sorted[:, -1])
    
    
    # Gap between first and second largest alignments for each neuron
    n_i = hbars_gamma_mean.shape[0]
    alignment_gaps = (hbars_gamma_mean[np.arange(n_i), specif_gammas]
                     - hbars_gamma_mean[np.arange(n_i), aligns_idx_sorted[:, -2]])
    
    # Variance (fluctuations) of hbars gamma in the last 20 minutes of the simul
    # Increases when the learning rate increases
    last_steps = int(2.0*duration_loc/3.0 / dt) // skp_loc
    hbars_gamma_vari = np.var(hbars_gamma[last_steps:], axis=0)
    
    return hbars_gamma, alignment_gaps, specif_gammas, hbars_gamma_vari

In [ ]:
def run_analyze_ibcm_one_back_seed(
        ibcm_rates_loc, back_rates, inhib_rates_loc, 
        options_loc, dimensions, seedseq, 
        duration_loc=360000.0, dt_loc=1.0, skp_loc=20, full_returns=False
    ):
    """ Given IBCM model rates and background parameters except
    background odors (but incl. number odors, c0), and a main seed sequence, 
    run and analyze convergence of IBCM on the background generated from that seed. 
    The seedseq should itself have been spawned from a root seed to have a distinct
    one per run; this still makes seeds reproducible yet distinct for different runs. 
    The seedseq here is spawned again for a background gen. seed and a simul. seed. 
    
    Args:
        dimensions: gives [n_components, n_dimensions, n_i_ibcm]
    
    Returns:
        iff full_return:
            gaps, specifs, hgamvari, hgammas_ser, sim_results
        else:
            gaps, specifs, hgamvari, None, None
        alignment_gaps: indexed [neuron]
        specif_gammas: indexed [neuron]
        gamma_vari: indexed [neuron, component]
    """
    #print("Initializing IBCM simulation...")
    # Get dimensions
    n_comp, n_dim, n_i = dimensions
    
    # Spawn back. generation seed and simul seed
    initseed, simseed = seedseq.spawn(2)
    
    # Duplicate back params before appending locally-generated odor vectors to them
    back_pms_loc = list(back_rates)
    
    # Create background
    rgen_init = np.random.default_rng(initseed)
    back_comps_loc = generate_odorant((n_comp, n_dim), rgen_init)
    back_comps_loc = back_comps_loc / l2_norm(back_comps_loc, axis=1)[:, None]

    # Add odors to the list of background parameters
    back_pms_loc.append(back_comps_loc)

    # Initialize background with the random generator with seed rgenseed
    rgen_init = np.random.default_rng(initseed)
    init_back = initialize_given_background(back_pms_loc, rgen_init, n_comp, n_dim)

    # Initial synaptic weights: small positive noise
    lambd_loc = ibcm_rates_loc[3]
    init_synapses_ibcm = 0.2*rgen_init.standard_normal(size=[n_i, n_dim])*lambd_loc
    
    # Run the IBCM simulation
    #print("Running IBCM simulation...")
    tstart = perf_counter()
    sim_results = integrate_inhib_ibcm_network_options(
                init_synapses_ibcm, update_fct, init_back, 
                ibcm_rates_loc, inhib_rates_loc, back_pms_loc, 
                duration_loc, dt_loc, seed=simseed, 
                noisetype="uniform",  skp=skp_loc, **options_loc
    )
    tend = perf_counter()
    #print("Finished IBCM simulation in {:.2f} s".format(tend - tstart))
    
    # Now analyze IBCM simul for convergence
    #print("Starting to analyze IBCM simulation...")
    tstart = perf_counter()
    hgammas_ser, gaps, specifs, hgamvari = analyze_ibcm_simulation(sim_results, 
                        ibcm_rates_loc, back_pms_loc, skp_loc=skp_loc, duration_loc=duration_loc)
    tend = perf_counter()
    #print("Finished analyzing IBCM simulation")
    
    # Doesn't return full c gamma series, only the summary statistics of convergence
    if full_returns:
        hgammas_ser_ret = hgammas_ser
        sim_results_ret = sim_results
    else:
        hgammas_ser_ret = None
        sim_results_ret = None
    
    return gaps, specifs, hgamvari, hgammas_ser_ret, sim_results_ret

# BioPCA simulation functions

In [ ]:
def run_biopca_simulation(biopca_rates_loc, back_params_loc, inhib_rates_loc, 
                        options_loc, dimensions, initseed, simseed, 
                        duration_loc=360000.0, dt_loc=1.0, skp_loc=20):
    """
        Args:
            biopca_rates_loc
            back_params_loc
            inhib_rates_loc: alpha, beta
            options_loc: model options
            dimensions: [n_components (n_b), n_dimensions (n_s), n_neurons (n_i)]
            initseed: int seed for background and weights initialization
            simseed: int seed for simulation
            
    """
    print("Initializing BioPCA simulation...")
    # Initialize background with the random generator with seed rgenseed
    n_comp, n_dim, n_i = dimensions
    
    # Create random generator with initseed for initial value choice
    rgen_init = np.random.default_rng(initseed)
    init_back = initialize_given_background(back_params_loc, rgen_init, n_comp, n_dim)

    # Initial synaptic weights: small positive noise
    init_synapses_pca = rgen_init.standard_normal(size=[n_i_pca, n_dim]) / np.sqrt(n_i_pca)
    init_mmat_pca = rgen_init.standard_normal(size=[n_i_pca, n_dim]) / np.sqrt(n_dim)
    init_lmat_pca = np.eye(n_i_pca, n_i_pca)  # Supposed to be near-identity, start as identity
    ml_inits_pca = [init_mmat_pca, init_lmat_pca]
    
    # Run the IBCM simulation
    print("Starting BioPCA simulation...")
    tstart = perf_counter()
    sim_results = integrate_inhib_biopca_network_skip(
                ml_inits_pca, update_fct, init_back, 
                biopca_rates_loc, inhib_rates_loc, back_params_loc, 
                duration_loc, dt_loc, seed=simseed, 
                noisetype="uniform",  skp=skp_loc, **options_loc
    )
    tend = perf_counter()
    print("Finished BioPCA simulation in {:.2f} s".format(tend - tstart))
    
    return sim_results

In [ ]:
def analyze_clean_biopca_simul(results_raw):
    raise NotImplementedError()

Enjoy a simplification for once: we do not need to consider other models like average subtraction, optimal $P$, orthogonal projection since all we care about in this notebook is the convergence of the two biologically plausible models. 

# Plotting functions

In [ ]:
# Plotting functions for IBCM
def plot_ibcm_results(res_ibcm_raw, hbars_gamma, skp=20):
    # tseries, bk_series, bkvec_series, m_series,
    # hbar_series, theta_series, w_series, y_series
    tser, bkser, bkvecser, _, _, _, _, yser = res_ibcm_raw
    tser_scaled = tser *  10.0 / 60.0  # in min
    # Plot of hbars gamma series
    fig , ax, _ = plot_hbars_gamma_series(tser_scaled, hbars_gamma, 
                            skp=skp, transient=320000 // skp)
    fig.tight_layout()
    leg = ax.legend(loc="upper left", bbox_to_anchor=(1., 1.))
    ax.set_xlabel("Time (min)")
    plt.show()
    plt.close()

    # Plot of background inhibition
    fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                    tser, bkvecser, yser, skp=skp)

    # Compute noise reduction factor, annotate
    transient = 250000 // skp
    norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

    print("Mean activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
    print("Standard deviation of activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
    ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
               xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

    ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
    fig.tight_layout()
    
    plt.show()
    plt.close()

# Simulation parameter choices

## Simulation parameters common to all runs

In [ ]:
# Common parameters for all simulations
# Dimensions: 25 is enough?
n_dimensions = 25
n_components = 4  # try with 3 for simplicity by default

# Inhibition W learning and decay rates
inhib_rates_default = [0.0001, 0.00002]  # alpha, beta  [0.00025, 0.00005]

# Simulation duration and integration time step
duration = 360000.0
deltat = 1.0

# Saving every skp simulation point, 50 is enough for plots, 
# here use 20 to get convergence time accurately
skp_default = 20 * int(1.0 / deltat)
tser_common = np.arange(0.0, duration, deltat*skp_default)

# Common model options
activ_function = "identity"  #"ReLU"

## IBCM default parameters

In [ ]:
# IBCM model parameters
n_i_ibcm = 24  # Number of inhibitory neurons for IBCM case

# Default model rates
learnrate_ibcm = 0.0005 #5e-5
tau_avg_ibcm = 1600  # 2000
coupling_eta_ibcm = 0.6/n_i_ibcm
ssat_ibcm = 50.0
k_c2bar_avg = 0.1
decay_relative_ibcm = 0.005
lambd_ibcm = 1.0
ibcm_rates_default = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm 
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "tanh", 
    "variant": "law", 
    "decay": True
}

In [ ]:
# Test run for now
# Create a default background for testing purposes
meta_seedseq = np.random.SeedSequence(0xa5ad7857493a6fc471b0283b8db69268)
#initial_seed = 0x36d8487b210fe0277493149915aba4f2
#simul_seed = 0xb1b9996e9f459ffc4ec9fc95c2db5583

# Package dimensions and back. parameters
back_rates_default = default_background_params(n_components)
# Try changing background rates here if desired
# Concentration scale c0: multiply up-down to change convergence dynamics
# just as well as the learning rate, although with a different scaling. 
back_rates_default[4][:] = 0.6
back_rates_default[1][:] = 500.0  # whiff duration
back_rates_default[3][:] = 800.0  # blank duration
dimensions_ibcm = [n_components, n_dimensions, n_i_ibcm]

#ibcm_rates_loc, back_rates, inhib_rates_loc, 
#        options_loc, dimensions, seedseq, 
#        duration_loc=360000.0, dt_loc=1.0, skp_loc=20, full_return=False

# Run and analyze simulation derived from the meta seedsequence
all_res = run_analyze_ibcm_one_back_seed(ibcm_rates_default, back_rates_default, inhib_rates_default, 
                        ibcm_options, dimensions_ibcm, meta_seedseq,
                        duration_loc=duration, dt_loc=deltat, skp_loc=skp_default, full_returns=True)

gaps, specifs, hgamvari, hgammas_ser, ibcm_results = all_res

In [ ]:
# Visualize convergence dynamics first
plot_ibcm_results(ibcm_results, hgammas_ser, skp=skp_default)

In [ ]:
# Print some other convergence analysis results
print("{} out of {} odors covered".format(np.unique(specifs).size, n_components))
print("Variance of the largest hbar_gamma for each neuron:\n", hgamvari[np.arange(n_i_ibcm), specifs])

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(n_i_ibcm), gaps, marker="o", mfc="w", ms=8)
for i in range(n_i_ibcm):
    ax.annotate(str(specifs[i]), xy=(i, gaps[i]), ha="center", va="center")
ax.set_ylim([0.0, gaps.max()*1.1])
ax.set(ylabel="Alignment gap", xlabel="Neuron", xticks=np.arange(0, n_i_ibcm, 3))
plt.show()

In [ ]:
mean_conc = np.mean(ibcm_results[1][:, :, 1])
conc_scale = back_rates_default[4].mean()
moments_conc = [
    mean_conc/conc_scale, 
    np.var(ibcm_results[1][:, :, 1])/conc_scale**2,
    np.mean((ibcm_results[1][:, :, 1] - mean_conc)**3.0)/conc_scale**3
]
print(moments_conc)

# Develop main simulations
For a grid of $\mu, \tau_\Theta$ choices, 

Later on, do the same for 3, 4, 5, 6, 7, 8 + odors

And do the same for different whiff and blank durations (scale both up or down?)

And do the same for the concentration amplitude, controlled by $c_0$, to show it is equivalent to scaling $\mu$?

Still need to figure out analytically which moment of the background (not just its scale) determines the time scale... 

In [ ]:
from threadpoolctl import threadpool_limits
from utils.cpu_affinity import count_parallel_cpu, count_threads_per_process
import multiprocessing
from simulfcts.idealized_recognition import func_wrapper_threadpool

#def func_wrapper_threadpool(func, threadlim, *args, **kwargs):
#    with threadpool_limits(limits=threadlim, user_api='blas'):
#        res = func(*args, **kwargs)
#    return res


In [ ]:
def combine_seed_results(res_dict, n_seeds):
    """ Combine convergence analysis results of simulation seeds
    in res_dict. """
    combi_gaps, combi_specifs, combi_varis = [], [], []
    for i in range(n_seeds):
        gaps, specifs, hgamvari, _, _ = res_dict[i]
        combi_gaps.append(gaps)
        combi_specifs.append(specifs)
        combi_varis.append(hgamvari)
    return [np.stack(a) for a in [combi_gaps, combi_specifs, combi_varis]]

In [ ]:
# This is a test version that does not really use multiprocessing
# because it doesn't work with functions defined in the notebook
# but the point is to develop functions for the full script in secondary_scripts/
def main_convergence_vs_ibcm_rates(orig_seedseq, n_seeds):
    """ Run n_seeds IBCM simulations for each combination of mu learning rate
    and tau_theta averaging time scale, collect convergence statistics for each. 
    The same simulation seeds are tested at each combinatino of model rates, 
    to assess the convergence vs these rates while keeping the set of
    backgrounds tested the same, for a more direct comparison. 
    
    Since this is a main function, the rates grid and other model parameters are
    defined within. 
    
    Args:
        orig_seedseq (np.random.SeedSequence): fresh SeedSequence from which
            all other simulation seeds will be spawned. 
    
    Returns:
        learnrate_tautheta_grid (np.ndarray): indexed [2, mu_idx, tau_idx]
            i.e. the first array along axis 0 contains the 2d grid of mu vals, 
            the second array contains the 2d grid of tau values, and in each grid,
            mu varies along axis 0 (rows), tau varies along axis 1 (columns); 
            the result of np.meshgrid(murange, taurange, indexing="ij"
        all_gaps (np.ndarray): indexed [mu, tau, seed, neuron]
        all_specifs (np.ndarray): indexed [mu, tau, seed, neuron]
        all_varis (np.ndarray): indexed [mu, tau, seed, neuron, component]
    """
    # Grid of IBCM rates mu, tau_theta in a range going to either side
    # of the region where we get convergence for N_B = 3 odors
    # Approximately geomspace, clustered around usual rates (0.75e-3 to 1.25e-3)
    learnrate_range = np.asarray([5e-5, 2e-4, 5e-4, 7.5e-4, 1.25e-3, 2e-3, 5e-3, 2e-2])[:2]
    # Also a somewhat geometric progression, clustered around good ones (800-1200-1600)
    tautheta_range = np.asarray([100, 200, 400, 800, 1200, 1600, 2000, 3000])[:2]
    learnrate_tautheta_grid = np.stack(
        np.meshgrid(learnrate_range, tautheta_range, indexing="ij"), axis=0)
    # learnrate varies along axis 0 (y, rows), tautheta along axis 1 (x, columns)
    
    # Define simulation and model parameters
    n_i_ibcm_sim = 24
    n_dims_sim = 25
    n_comp_sim = 3
    dimensions_sim = [n_comp_sim, n_dims_sim, n_i_ibcm_sim]
    
    # Default IBCM model rates
    learnrate_ibcm_sim = 0.00125  # will vary
    tau_avg_ibcm_sim = 1200  # will vary
    coupling_eta_ibcm_sim = 0.6/n_i_ibcm_sim
    ssat_ibcm_sim = 50.0
    k_c2bar_avg_sim = 0.1
    decay_relative_ibcm_sim = 0.005
    lambd_ibcm_sim = 1.0
    ibcm_rates_sim = [
        learnrate_ibcm_sim, 
        tau_avg_ibcm_sim, 
        coupling_eta_ibcm_sim, 
        lambd_ibcm_sim,
        ssat_ibcm_sim, 
        k_c2bar_avg_sim,
        decay_relative_ibcm_sim
    ]
    ibcm_options_sim = {
        "activ_fct": "identity",
        "saturation": "tanh", 
        "variant": "law",   # maybe we will want to test "intrator" later?
        "decay": True
    }
    # default turbulent background parameters
    back_rates_sim = default_background_params(n_comp_sim)
    # Default alpha, beta
    inhib_rates_sim = [0.0001, 0.00002]  # alpha, beta
    
    # Time parameters
    duration_sim = 36000.0
    deltat_sim = 1.0
    skp_sim = 20
    
    # Containers for alignment gaps, specificities, and hgammas variances
    # that will be stacked arrays indexed [mu, tau, seed, ...]
    all_gaps, all_specifs, all_varis = [], [], []
    
    # Spawn simulation seeds, reused at each combination on the IBCM rates grid
    simul_seeds = orig_seedseq.spawn(n_seeds)
    n_workers = min(count_parallel_cpu(), n_seeds)
    n_threads = count_threads_per_process(n_workers)
    pool = multiprocessing.Pool(n_workers)

    # Treat one rate combination at a time
    for i in range(learnrate_range.shape[0]):
        mu = learnrate_range[i]
        i_gaps, i_specifs, i_varis = [], [], []
        for j in range(tautheta_range.shape[0]):
            tau = tautheta_range[j]
            ibcm_rates_sim[0] = mu
            ibcm_rates_sim[1] = tau
            # Launch multiple seeds for the current (mu, tau) combination
            all_procs_mutau = {}
            res_seeds_mutau = {}
            for k in range(n_seeds):
                apply_args = (run_analyze_ibcm_one_back_seed, n_threads, 
                              ibcm_rates_sim, back_rates_sim, inhib_rates_sim,  
                              ibcm_options_sim,dimensions_sim, simul_seeds[k])
                apply_kwds = dict(duration_loc=duration_sim, dt_loc=deltat_sim, 
                                  skp_loc=skp_sim, full_returns=False)
                #all_procs_mutau[k] = pool.apply_async(func_wrapper_threadpool, 
                #                 args=apply_args, kwds=apply_kwds)
                res_seeds_mutau[k] = func_wrapper_threadpool(*apply_args, **apply_kwds)
                

            # Collect convergence analysis results for this mu, tau
            #res_seeds_mutau = {k:all_procs_mutau[k].get() for k in all_procs_mutau.keys()}
            # Stack them over seeds
            combined_seed_res = combine_seed_results(res_seeds_mutau, n_seeds)
            i_gaps.append(combined_seed_res[0])
            i_specifs.append(combined_seed_res[1])
            i_varis.append(combined_seed_res[2])
            print("Finished mu i = {}, tau j = {}".format(i, j))
        
        # Stack arrays over j (tau_theta) for the current i value (mu)
        all_gaps.append(np.stack(i_gaps))
        all_specifs.append(np.stack(i_specifs))
        all_varis.append(np.stack(i_varis))
    
    # Stack arrays over i (mu)
    all_gaps = np.stack(all_gaps)
    all_specifs = np.stack(all_specifs)
    all_varis = np.stack(all_varis)
    
    # Reuse pool for each (mu, tau) but close them at the end
    pool.close()
    pool.join()
    
    return learnrate_tautheta_grid, all_gaps, all_specifs, all_varis
    

In [ ]:
#mutau_grid, align_gaps, specifs, varis = main_convergence_vs_ibcm_rates(
#    np.random.SeedSequence(0xf44f62d0818452d631061e695b75c517), 2)

# Load complete simulations run on the cluster
conv_results = np.load(pj(outputs_folder, "convergence_vs_ibcm_rates_results.npz"))

In [ ]:
mutau_grid = conv_results["mutau_grid"]
align_gaps = conv_results["align_gaps"]
gamma_specifs = conv_results["gamma_specifs"]
hgamma_varis = conv_results["hgamma_varis"]

In [ ]:
n_mu, n_tau = mutau_grid.shape[1:3]

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau-1, -1, -1):
    gap_mean_line_tau = np.mean(align_gaps[:, j, :], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    gap_std_line_tau = np.std(np.mean(align_gaps[:, j, :], axis=2), axis=1, ddof=1)
    ax.plot(mu_range, gap_mean_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    ax.fill_between(mu_range, gap_mean_line_tau-gap_std_line_tau, 
                   gap_mean_line_tau+gap_std_line_tau, color=colors[j], alpha=0.15)
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Alignment gap\n(larger is better)", xscale="log")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, title=r"$\tau_{\Theta}$", 
          loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots()
tau_range = mutau_grid[1, 0, :]
colors = sns.color_palette("ocean", n_colors=n_tau)
for i in range(n_mu-1, -1, -1):
    gap_mean_line_mu = np.mean(align_gaps[i, :, :], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    gap_std_line_mu = np.std(np.mean(align_gaps[i, :, :], axis=2), axis=1, ddof=1)
    ax.plot(tau_range, gap_mean_line_mu, color=colors[i], label=mutau_grid[0, i, 0])
    ax.fill_between(tau_range, gap_mean_line_mu-gap_std_line_mu, 
                   gap_mean_line_mu+gap_std_line_mu, color=colors[i], alpha=0.1)
ax.set(xlabel=r"Averaging time scale $\tau_{\Theta}$", 
       ylabel="Alignment gap\n(larger is better)", xscale="log")
ax.legend(title=r"$\mu$", loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()

In [ ]:
align_gaps.shape

In [ ]:
# Coverage of odors: fraction of seeds where all 3 odors are covered?
fig, ax = plt.subplots()
# extent: left, right, bottom, top
#im = ax.imshow(np.mean(align_gaps, axis=(2, 3)).T, 
#          extent=(mu_range.min(), mu_range.max(), tau_range.min()*0.01, tau_range.max()*0.01), 
#         cmap="viridis", aspect="auto")
im = ax.pcolormesh(mutau_grid[0], mutau_grid[1], np.mean(align_gaps, axis=(2, 3)), cmap="viridis")
ax.set(xlabel=r"Learning rate $\mu$", ylabel=r"Averaging time $\tau_{\Theta}$ (s)", xscale="log")
ax.set_xlim(mu_range[0], mu_range[-1])
fig.colorbar(im, label="Alignment gap")
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots()
colors = sns.color_palette("magma", n_colors=n_tau)
# Plot the variance of the specific c_gamma, mean across neurons and seeds
stdev_specifs = np.zeros(hgamma_varis.shape[:4])
n_i = gamma_specifs.shape[3]
for i in range(n_mu):
    for j in range(n_tau):
        for k in range(hgamma_varis.shape[2]):
            stdev_specifs[i, j, k] = hgamma_varis[i, j, k][np.arange(n_i), gamma_specifs[i, j, k]]
        
for j in range(n_tau):
    std_mean_line_tau = np.mean(stdev_specifs[:, j, :], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    std_std_line_tau = np.std(stdev_specifs[:, j, :], axis=(1, 2), ddof=1)
    ax.plot(mu_range, std_mean_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    #ax.fill_between(mu_range, std_mean_line_tau-std_std_line_tau, 
    #               std_mean_line_tau+std_std_line_tau, color=colors[j], alpha=0.15)
ax.set(xlabel=r"Learning rate $\mu$", ylabel=r"Standard dev. of $h_{\gamma,sp}$" "\n(smaller is better)", 
       xscale="log", yscale="log")
ax.legend(title=r"$\tau_{\Theta}$", ncol=2, frameon=False)
plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots()
colors = sns.color_palette("magma", n_colors=n_tau)
# Plot the variance of the specific c_gamma, mean across neurons and seeds
n_i = gamma_specifs.shape[3]
for i in range(n_mu-1, -1, -1):
    std_mean_line_mu = np.mean(stdev_specifs[i, :, :], axis=(1, 2))
    # Variance across seed of the mean alignment in a simulation. 
    # We don't wan't the intra-simulation variance, neurons selecting different 
    # odors may converge to different alignment values due to background fluctuations
    std_std_line_mu = np.std(stdev_specifs[i, :, :], axis=(1, 2), ddof=1)
    ax.plot(tau_range, std_mean_line_mu, color=colors[i], label=mutau_grid[0, i, 0])
    #ax.fill_between(tau_range, std_mean_line_mu-std_std_line_mu, 
    #               std_mean_line_mu+std_std_line_mu, color=colors[i], alpha=0.15)
ax.set(xlabel=r"Averaging time $\tau_\Theta$", 
       ylabel=r"Standard dev. of $h_{\gamma,sp}$" "\n(smaller is better)", yscale="log")
ax.legend(title=r"$\mu$", frameon=False, bbox_to_anchor=(1, 1))
plt.show()
plt.close()

In [ ]:
# TODO: ascertain plot vs tau

In [ ]:
# Coverage of odors: fraction of seeds where all 3 odors are covered?
odor_coverage = np.zeros(gamma_specifs.shape[:3])
for i in range(n_mu):
    for j in range(n_tau):
        for k in range(gamma_specifs.shape[2]):
            odor_coverage[i, j, k] = np.unique(gamma_specifs[i, j, k]).size
odor_coverage_mean = np.mean(odor_coverage, axis=2)

fig, ax = plt.subplots()
# extent: left, right, bottom, top
#im = ax.imshow(odor_coverage_mean.T, 
#          extent=(mu_range.min(), mu_range.max(), tau_range.min()*0.01, tau_range.max()*0.01), 
#         cmap="viridis", aspect="auto")
im = ax.pcolormesh(mutau_grid[0], mutau_grid[1], odor_coverage_mean, cmap="viridis")
ax.set(xlabel=r"Learning rate $\mu$", ylabel=r"Averaging time $\tau_{\Theta}$ (s)", xscale="log", yscale="log")
ax.set_xlim(mu_range[0], mu_range[-1])
ax.set_yticks(tau_range)
fig.colorbar(im, label="Average # odors covered")
plt.show()
plt.close()

In [ ]:
n_mu, n_tau = mutau_grid.shape[1:3]

fig, ax = plt.subplots()
mu_range = mutau_grid[0, :, 0]
colors = sns.color_palette("magma", n_colors=n_tau)
for j in range(n_tau):
    coverage_line_tau = odor_coverage_mean[:, j]
    coverage_std_tau = np.std(odor_coverage[:, j], axis=1)  # std across seed
    #ax.plot(mu_range, coverage_line_tau, color=colors[j], label=int(mutau_grid[1, 0, j]))
    ax.plot(mu_range, np.mean(odor_coverage[:, j], axis=1), color=colors[j], label=int(mutau_grid[1, 0, j]))
    #ax.fill_between(mu_range, coverage_line_tau-coverage_std_tau, 
    #               coverage_line_tau+coverage_std_tau, color=colors[j], alpha=0.0)
ax.set(xlabel=r"Learning rate $\mu$", ylabel="Average # odors covered", xscale="log")
ax.legend(title=r"$\tau_{\Theta}$", loc="upper left", bbox_to_anchor=(1, 1), frameon=False)
plt.show()
plt.close()